# Advent of code 2018

Lets start with necessary imports

In [1]:
import urllib.request as urllib2  # the lib that handles the url stuff
from datetime import datetime
from datetime import timedelta
import operator
from collections import Counter
import numpy as np
import scipy.misc as smp

### Day 1

The first part requires us to get a sum of numbers found in a txt file.

In [2]:
fname = "day/1/input.txt"
with open(fname) as f:
    content = f.readlines()
day_1_content = [int(i) for i in content]
print("Sum of numbers is:")
print(sum(day_1_content))

Sum of numbers is:
406


In the second part, we have to keep summing the numbers found in the text file until we have found the first sum repeated twice

In [3]:
def find_reoccurring_frequency(number_list):
    not_found = True
    current_sum = 0
    sum_history = {}
    while(not_found):
        for number in number_list:
            current_sum += number
            if (current_sum in sum_history):
                print("First sum found twice is:")
                print(current_sum)
                not_found = False
                break
            else:
                sum_history[current_sum] = 1

find_reoccurring_frequency(day_1_content)

First sum found twice is:
312


### Day 2

The first part requires us to get find all entries of text blocks that have 2 or 3 of the same character. Multiplying the count of these gives the result.

In [4]:
fname = "day/2/input.txt"
with open(fname) as f:
    content = f.readlines()
day_2_content = [i.rstrip() for i in content]
#print(day_2_content)

def find_checksum(content):
    double = 0
    triple = 0
    for s in content:
        tmp = {}
        for letter in s:
            if letter in tmp:
                tmp[letter] += 1
            else:
                tmp[letter] = 1
        if (2 in tmp.values()):
            double += 1
        if (3 in tmp.values()):
            triple += 1
    print (double*triple)

def find_diff_of_one(content):
    for word1 in content:
        finished = False
        for word2 in content:
            differences = 0
            for i in range(len(word1)):
                if (word1[i]!=word2[i]):
                    differences+=1
            if (differences == 1):
                common = ""
                for i in range(len(word1)):
                    if (word1[i]==word2[i]):
                        common += word1[i]
                print(common)
                finished = True
                break
        if (finished):
            break

find_checksum(day_2_content)
find_diff_of_one(day_2_content)

5390
nvosmkcdtdbfhyxsphzgraljq


### Day 4


In [5]:
fname = "day/4/input.txt"
with open(fname) as f:
    content = f.readlines()
day_4_content = [i.rstrip() for i in content]


def get_sorted_entries(content):
    entrylist = []
    for entrystring in day_4_content:
        date = datetime.strptime(entrystring.split(']')[0][1:], '%Y-%m-%d %H:%M')
        entrylist.append({"time":date, "msg" : entrystring.split(']')[1]})
    sortedentries = sorted(entrylist, key=lambda r: r['time'])
    return sortedentries

def generate_guard_history(sortedentries):
    guard_history = {}
    guard = ""
    sleepytime = 0
    for entry in sortedentries:
        if "Guard" in entry['msg']:
            guard = entry['msg'].split('#')[1].split(' ')[0]
        if "falls asleep" in entry['msg']:
            sleepytime = entry['time']
        if "wakes" in entry['msg']:
            if not guard in guard_history:
                guard_history[guard] = {"total":0, "times":{}}
            while sleepytime < entry['time']:
                guard_history[guard]["total"] += 1
                clock = sleepytime.strftime("%M")
                if (clock in guard_history[guard]["times"]):
                    guard_history[guard]["times"][clock] +=1
                else:
                    guard_history[guard]["times"][clock]=0
                sleepytime += timedelta(minutes=1)
    return guard_history

def get_max_sleeper_times_max_minute(guard_history):
    maxim = 0
    maxguard = ""
    for guard in guard_history.items():
        if guard[1]['total'] > maxim:
            maxim = guard[1]['total']
            maxguard = guard[0]
    maxtimes = guard_history[maxguard]["times"]
    max_key = max(maxtimes, key=lambda k: maxtimes[k])
    return int(maxguard)*int(max_key)

def get_most_frequent_sleep_minute_times_sleeper(guard_history):
    maxim = 0
    maxmin = 0
    maxguard = 0
    for guard in guard_history.items():
        for time in guard[1]["times"].items():
            if (time[1] > maxim):
                maxim = time[1]
                maxguard = guard[0]
                maxmin = time[0]
    return int(maxmin)*int(maxguard)        

sortedentries = get_sorted_entries(day_4_content)
guard_history = generate_guard_history(sortedentries)
part1_ans = get_max_sleeper_times_max_minute(guard_history)
print(part1_ans)
part2_ans = get_most_frequent_sleep_minute_times_sleeper(guard_history)
print(part2_ans)




    

72925
49137


### Day 5

In [6]:
fname = "day/5/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_5_content = content[0].rstrip()

alphabet = "abcdefghijklmnopqrstuvwxyz"
reactable = [c + c.upper() for c in alphabet]
reactable += [c.upper() + c for c in alphabet]

reacting = True
def reaction_start(polymer):
    while(True):
        old = polymer
        for pair in reactable:
            polymer = polymer.replace(pair, '')
        if (polymer == old):
            break
    return polymer
print(len(reaction_start(day_5_content)))


results = []
for c in alphabet:
    test_str = day_5_content.replace(c, '')    
    test_str = test_str.replace(c.upper(), '')
    results.append((c, len(reaction_start(test_str))))
print(min(results, key = lambda t: t[1]))

10886
('v', 4684)


### Day 6


In [ ]:
def manhattan_dist(x, x2, y, y2):
    return (abs(x-x2)+abs(y-y2))

def visualizer(array, x, y, name):
    with open("day/6/"+name, "w") as the_file:
        for i in range(x):
            line = ""
            for j in range(y):
                dist = array[i,j]["dist"]
                if dist==0:
                    line+="0"
                else:
                    line+=(array[i,j]["closest"])
            line+=("\n")
            the_file.write(line)
        
def draw_image(array, x, y, identifiers):
    colors = {}
    for idt in identifiers:
        color = list(np.random.choice(range(256), size=3))
        colors[idt] = color
    colors['.'] = [255, 255, 255]

    data = np.zeros( (x, y, 3), dtype=np.uint8 )
    for i in range(x):
        for j in range(y):
            data[i,j] = colors[array[i,j]["closest"]]
    img = smp.toimage(data)
    img.show()

fname = "day/6/input.txt"
with open(fname) as f:
    content = f.readlines()
day_6_content = [i.rstrip().split(', ') for i in content]
day_6_content = [[int(i[0]), int(i[1])]  for i in day_6_content]
x_len = max(day_6_content, key=lambda item:item[0])[0]
y_len = max(day_6_content, key=lambda item:item[1])[1]
#print(x_len)
#print(y_len)
#print(day_6_content)
array = {}
for i in range(x_len):
    for j in range(y_len):
        array[i, j] = {"closest":"", "dist":360}

identifiers = "abcdefghijklmnopqrstuvwxyABCDEFGHIJKLMNOPQRSTUVWXY"
        
for idx, coord in enumerate(day_6_content):
    x = coord[0]
    y = coord[1]
    for i in range(x_len):
        for j in range(y_len):
            dist = manhattan_dist(x, i, y, j)
            if (dist<array[i,j]["dist"]):
                array[i,j]["dist"] = dist
                array[i,j]["closest"] = identifiers[idx]
            elif (dist==array[i,j]["dist"]):
                array[i,j]["closest"] = '.'


visualizer(array, x_len, y_len, 'test_print.txt')
draw_image(array, x_len, y_len, identifiers)
infinite = []
for i in range(x_len):
    a = array[i, 0]["closest"]
    b = array[i, y_len-1]["closest"]
    if a not in infinite:
        infinite.append(a)
    if b not in infinite:
        infinite.append(b)
for j in range(y_len):
    a = array[0, j]["closest"]
    b = array[x_len -1, j]["closest"]
    if a not in infinite:
        infinite.append(a)
    if b not in infinite:
        infinite.append(b)

#print(infinite)
counts = Counter()
for i in range(x_len):
    for j in range(y_len):
        val = array[i, j]["closest"]
        if val not in infinite:
            counts.update(str(val))
maxim = counts.most_common()[0]
print(maxim)

safe_array = {}
for i in range(x_len):
    for j in range(y_len):
        safe_array[i, j] = {"closest":"#", "dist":10000}
for idx, coord in enumerate(day_6_content):
    x = coord[0]
    y = coord[1]
    for i in range(x_len):
        for j in range(y_len):
            dist = manhattan_dist(x, i, y, j)
            if (safe_array[i,j]['closest']=='#'):
                safe_array[i,j]['dist']-=dist
                if(safe_array[i,j]['dist']<=0):
                    safe_array[i,j]['closest']='.'
visualizer(safe_array, x_len, y_len, 'test_print2.txt')
safecount = 0
for i in range(x_len):
    for j in range(y_len):
        if safe_array[i,j]['closest']=='#':
            safecount+=1
print(safecount)
draw_image(safe_array, x_len, y_len, "#.")